kaggle 叶子分类问题 Simple Kears 1D CNN
输入数据在上一层的leaf_classification里 train.csv(991条)和test.csv（595条），number_class=99
在csv文件中，特征集的保存方式nxm，其中n为样本个数，192个特征。这里有3类特征，64个magin特征， 64个shape特征， 64个texture特征。

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

train = pd.read_csv('../leaf_classification/train.csv')
test = pd.read_csv('../leaf_classification/test.csv')
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

nb_features = 64 # number of features per features type (shape, texture, margin) 
nb_class = len(classes)

# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
#model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Convolution1D(filters=512, kernel_size=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(learning_rate=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, epochs=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

Epoch 1/15
56/56 [==============================] - 23s 382ms/step - loss: 3.6651 - accuracy: 0.2458 - val_loss: 1.4632 - val_accuracy: 0.6970
Epoch 2/15
56/56 [==============================] - 21s 380ms/step - loss: 0.7881 - accuracy: 0.8395 - val_loss: 0.4835 - val_accuracy: 0.9091
Epoch 3/15
56/56 [==============================] - 21s 381ms/step - loss: 0.1401 - accuracy: 0.9675 - val_loss: 0.2206 - val_accuracy: 0.9192
Epoch 4/15
56/56 [==============================] - 21s 379ms/step - loss: 0.0303 - accuracy: 0.9921 - val_loss: 0.1704 - val_accuracy: 0.9394
Epoch 5/15
56/56 [==============================] - 21s 380ms/step - loss: 0.0210 - accuracy: 0.9944 - val_loss: 0.1975 - val_accuracy: 0.9495
Epoch 6/15
56/56 [==============================] - 21s 380ms/step - loss: 0.0292 - accuracy: 0.9955 - val_loss: 0.1194 - val_accuracy: 0.9596
Epoch 7/15
56/56 [==============================] - 21s 380ms/step - loss: 0.0251 - accuracy: 0.9944 - val_loss: 0.1803 - val_accuracy: 0.9495

99